In [3]:
import asyncio
import json
from datetime import timedelta
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession
from mcp.types import CallToolResult, TextContent
from typing import Any, Dict, List, Optional, Tuple
class VenusFactoryClient:
    
    def __init__(self, server_url: str="https://scp.intern-ai.org.cn/api/v1/mcp/1/VenusFactory"):
        self.server_url = server_url
        self.session = None
        
    async def connect(self, timeout: int = 60):
        print(f"server url: {self.server_url}")
        try:
            self.transport = streamablehttp_client(
                url=self.server_url, 
                headers={"SCP-HUB-API-KEY": "sk-a0033dde-b3cd-413b-adbe-980bc78d6126"}
            )
            
            self.read, self.write, self.get_session_id = await asyncio.wait_for(
                self.transport.__aenter__(), 
                timeout=timeout
            )
            
            self.session_ctx = ClientSession(self.read, self.write)
            self.session = await self.session_ctx.__aenter__()
            
            await asyncio.wait_for(
                self.session.initialize(), 
                timeout=timeout
            )
            session_id = self.get_session_id()
            
            print(f"✓ connect success")
            return True
            
        except asyncio.TimeoutError:
            print(f"✗ connect timeout (>{timeout}s)")
            return False
        except Exception as e:
            print(f"✗ connect failed: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    async def disconnect(self):
        try:
            if self.session:
                await self.session_ctx.__aexit__(None, None, None)
            if hasattr(self, 'transport'):
                await self.transport.__aexit__(None, None, None)
            print("✓ already disconnect")
        except Exception as e:
            print(f"✗ disconnect error: {e}")
    
    async def list_tools(self):
        try:
            tools_list = await self.session.list_tools()
            print(f"tool count: {len(tools_list.tools)}")
            
            for i, tool in enumerate(tools_list.tools, 1):
                print(f"{i:2d}. {tool.name}")
                if tool.description:
                    # 只显示描述的第一行
                    desc_line = tool.description.split('\n')[0]
                    print(f"    {desc_line}")
            
            print(f"\n✓ get tool list success")
            return tools_list.tools
            
        except Exception as e:
            print(f"✗ get tool list failed: {e}")
            return []
            
    def parse_result(self, result: Any) -> Any:
        """
        Parse result from session response and extract data.
        
        Returns:
            - For successful responses: returns the 'data' field directly
            - For error responses: raises an exception with the error message
            - For non-standard responses: returns the raw content
        """
        if isinstance(result, CallToolResult):
            payloads: List[Any] = []
            for item in result.content or []:
                if isinstance(item, TextContent):
                    text = (item.text or "").strip()
                    if not text:
                        continue
                    try:
                        parsed = json.loads(text)
                        # Check if it's a standard MCP response format
                        if isinstance(parsed, dict):
                            # If it has the success/data/error structure
                            if "success" in parsed:
                                if parsed["success"]:
                                    # Return the data field directly for successful responses
                                    return parsed.get("data")
                                else:
                                    # Raise exception for error responses
                                    error_info = parsed.get("error", {})
                                    error_msg = error_info.get("message", "Unknown error")
                                    error_code = error_info.get("code", "ERROR")
                                    raise Exception(f"[{error_code}] {error_msg}")
                            # Otherwise return the parsed dict as-is
                            return parsed
                        return parsed
                    except json.JSONDecodeError:
                        payloads.append(text)
                elif getattr(item, "type", None) == "application/json" and hasattr(item, "data"):
                    payloads.append(item.data)
            if len(payloads) == 1:
                return payloads[0]
            if payloads:
                return payloads
        return result



In [ ]:
'''VenusFactory Example:
The UniProt ID of protein catalase is P04040. First, its protein sequence information is retrieved from UniProt. 
Next, a fine-tuned protein language model is employed to predict its solubility. Building upon this, active site prediction is performed to identify potentially critical functional residues. 
Finally, a protein mutation design tool is used to conduct zero-shot mutation evaluation and design, generating single-point mutations that may enhance protein performance, thereby providing actionable insights for subsequent experimental validation and engineering optimization.
'''
async def main():
    client = VenusFactoryClient()
    if not await client.connect():
        print("connection failed")
        return
    
    try:
        print("\nStep 1: Fetching protein sequence from UniProt...")
        result = await client.session.call_tool(
            "query_uniprot",
            arguments={"uniprot_id": "P04040"}
        )
        result_data = client.parse_result(result)
        
        # Extract sequence from MCP response format
        sequence = ""
        if isinstance(result_data, dict):
            if result_data.get("success") and "data" in result_data:
                data = result_data["data"]
                if isinstance(data, dict):
                    sequence = data.get("sequence", "")
                elif isinstance(data, str):
                    sequence = data
            elif "sequence" in result_data:
                sequence = result_data["sequence"]
        else:
            sequence = str(result_data) if result_data else ""
        
        if not sequence:
            print("✗ Failed to retrieve sequence")
            return
            
        print(f"✓ Sequence retrieved: {sequence[:50]}..." if len(str(sequence)) > 50 else f"✓ Sequence: {sequence}")
        
        print("\nStep 2: Predicting protein solubility...")
        result = await client.session.call_tool(
            "predict_protein_function",
            arguments={"sequence": sequence, "model_name": "ProtT5-xl-uniref50", "task": "Solubility"}
        )
        print(result)
        result_data = client.parse_result(result)
        print(result_data)
        print("\nStep 3: Predicting activity site...")
        result = await client.session.call_tool(
            "predict_functional_residue",
            arguments={"sequence": sequence, "model_name": "ProtT5-xl-uniref50", "task": "Binding Site"}
        )
        print(result)
        result_data = client.parse_result(result)
        print(result_data)
        print("\nStep 4: Protein mutation zero-shot prediction...")
        result = await client.session.call_tool(
            "predict_zero_shot_sequence",
            arguments={"sequence": sequence, "model_name": "ESM-1v"}
        )
        print(result)
        result_data = client.parse_result(result)
        print(result_data)
        return result_data
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return None
    
    finally:
        await client.disconnect()


if __name__ == '__main__':
    await main()

server url: https://scp.intern-ai.org.cn/api/v1/mcp/1/VenusFactory
✓ connect success

Step 1: Fetching protein sequence from UniProt...
✓ Sequence retrieved: MADSRDPASDQMQHWKEQRAAQKADVLTTGAGNPVGDKLNVITVGPRGPL...

Step 2: Predicting protein solubility...
meta=None content=[TextContent(type='text', text='{\n  "success": true,\n  "timestamp": "2025-12-24T11:23:20.509175Z",\n  "request_id": "5d763e88-2551-4706-9f09-3a4d1b2dc0a0",\n  "data": "{\'headers\': [\'Protein Name\', \'Sequence\', \'Predicted Class\', \'Confidence Score\'], \'data\': [[\'temp_sequence\', \'MADSRDPASDQMQHWKEQRAAQKADVLTTGAGNPVGDKLNVITVGPRGPLLVQDVVFTDEMAHFDRERIPERVVHAKGAGAFGYFEVTHDITKYSKAKVFEHIGKKTPIAVRFSTVAGESGSADTVRDPRGFAVKFYTEDGNWDLVGNNTPIFFIRDPILFPSFIHSQKRNPQTHLKDPDMVWDFWSLRPESLHQVSFLFSDRGIPDGHRHMNGYGSHTFKLVNANGEAVYCKFHYKTDQGIKNLSVEDAARLSQEDPDYGIRDLFNAIATGKYPSWTFYIQVMTFNQAETFPFNPFDLTKVWPHKDYPLIPVGKLVLNRNPVNYFAEVEQIAFDPSNMPPGIEASPDKMLQGRLFAYPDTHRHRLGPNYLHIPVNCPYRARVANYQRDGPMCMQDNQGGAPNYYPNSFGAPEQQPSALEHSIQYSGEVRRFNT